<a href="https://colab.research.google.com/github/petcheme/fisica/blob/desarrollo/PaquetesGaussianos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Idea General
Este cuaderno está armado para jugar con la propagación de un pulso gaussiano para los dos casos de interés:
- medios dispersivos
- medios no dispersivos

donde se toma que $$w(k) = w(k_0) + w'(k_0)(k - k_0) + \frac{w''(k_0)}{2}(k-k_0)^{2}$$
es una buena aproximación para la relación de dispersión en el caso dispersivo.

Como se ha visto previamente, $$w'(k_0) = v_g$$, la velocidad de grupo, y, haciendo una analogía, $$w''(k_0) = a_g$$, la variación de la velocidad de grupo como función de $$k$$.

In [1]:
"""
Traemos las librerias
"""

import numpy as np
import matplotlib.pylab as plt
from matplotlib import animation, rc
rc('animation', html='html5') #para poder ver la animacion en formato inline
from IPython.display import HTML

In [97]:
"""
Definimos los desplazamientos
"""

def desplazamiento_no_dispersivo(x,t,k0,dk,c):
    return 2*dk*np.cos(k0 * (x + c * t))*np.e**(-((x + c * t)**2)*dk**2)

def desplazamiento_dispersivo(x,t,k0,w0,dk,c,v_g,a_g):

    desplazamiento_dispersivo = [] # Tuve que hacer esto para poder usar el análisis complejo de python (seguramente se pueda implementar de otra forma)
    for xs in x:
        alpha = np.sqrt(complex(1 / 4 / dk ** 2, a_g(k0,w0) * t / 2))
        beta = (xs + v_g(k0,w0) * t) / 2
        desplazamiento_dispersivo.append(np.e ** (complex(0, k0 * (xs + c * t))) * np.e ** (- beta ** 2 / alpha ** 2) / alpha)
    return [ds.real for ds in desplazamiento_dispersivo]

In [50]:
"""
Tiempo y posición (notar que el espacio es negativo porque hice las cuentas considerando que viaja hacia la izquierda)

"""
x = np.arange(-20,0,0.01)
t = np.arange(0,5,0.01)


In [124]:
"""
Definimos las variables

Perdiendo un poco de generalidad pero para encontrar algo físicamente relevante, se trabaja con una relación de dispersión tipo plasma

w^2 = c^2 + w_0^2

"""
w0 = 50
k0 = 10 # número de onda en el cual se centra el paquete
dk = 1 # ancho del paquete
c = 5 # velocidad de fase (w_0/k_0 en el caso dispersivo)
v_g = lambda k,w : c * (1 + (w / k / c) ** 2) ** (-1/2) # velocidad de grupo (w' para dispersivo)
a_g = lambda k,w : c ** 2 * (1 + (k * c / w) ** 2) ** (-3/2) / w # aceleración de grupo (w'' para dispersivo)

In [125]:
"""
Las variables a simular
"""

var_1 = np.zeros((len(t), len(x)))
var_2 = np.zeros((len(t), len(x)))
for i, ts in enumerate(t):    
    var_1[i,:] = desplazamiento_no_dispersivo(x, ts, k0, dk, c)
    var_2[i,:] = desplazamiento_dispersivo(x,ts,k0,w0,dk,c,v_g,a_g)

In [ ]:
"""
Transformamos la simulación en una animación
"""

fig, ax = plt.subplots()

ax.set_title('Paquetes Gaussianos')
ax.set_xlim(x[0], x[-1]);
ax.set_ylim(-2.5, 2.5);
ax.set_xlabel('Posición');
ax.set_ylabel('Desplazamiento del Equilibrio');
line1, = ax.plot([], [], label = 'No Dispersivo')
line2, = ax.plot([], [], label = 'Dispersivo')
plt.legend(loc = 'upper left')


def animate(i):
    X = x
    Y = var_1[i,:]
    line1.set_data(X,Y)
    Y = var_2[i,:]
    line2.set_data(X,Y)
    return (line1,line2)
  
anim2 = animation.FuncAnimation(fig, animate, frames=len(t), interval=np.diff(t)[0]*5000)
anim2